In [ ]:
import cupy as cp
import numpy as np
from time import perf_counter
import time
from pyqcu import define
from pyqcu import io
from pyqcu import qcu
import cupyx.scipy.sparse.linalg as csla
import scipy.linalg as sla

print('My rank is ', define.rank)

In [ ]:
params = np.array([0]*define._PARAMS_SIZE_, dtype=np.int32)
params[define._LAT_X_] = 32
params[define._LAT_Y_] = 32
params[define._LAT_Z_] = 32
params[define._LAT_T_] = 32
params[define._LAT_XYZT_] = 1048576
params[define._GRID_X_] = 1
params[define._GRID_Y_] = 1
params[define._GRID_Z_] = 1
params[define._GRID_T_] = 1
params[define._PARITY_] = 0
params[define._NODE_RANK_] = 0
params[define._NODE_SIZE_] = 1
params[define._DAGGER_] = 0
params[define._MAX_ITER_] = 1e4
params[define._DATA_TYPE_] = 0
params[define._SET_INDEX_] = 2
params[define._SET_PLAN_] = 0
argv = np.array([0.0]*define._ARGV_SIZE_, dtype=np.float32)
argv[define._MASS_] = 0.0
argv[define._TOL_] = 1e-9
print("Parameters:", params)
print("Arguments:", argv)

In [ ]:
gauge_filename = f"quda_wilson-dslash-gauge_-{params[define._LAT_X_]}-{params[define._LAT_Y_]}-{params  [define._LAT_Z_]}-{params[define._LAT_T_]}-{params[define._LAT_XYZT_]}-{params[define._GRID_X_]}-{params[define._GRID_Y_]}-{params[define._GRID_Z_]}-{params[define._GRID_T_]}-{params[define._PARITY_]}-{params[define._NODE_RANK_]}-{params[define._NODE_SIZE_]}-{params[define._DAGGER_]}-f.bin"
print("Gauge filename:", gauge_filename)
gauge = cp.fromfile(gauge_filename, dtype=cp.complex64,
                    count=params[define._LAT_XYZT_]*define._LAT_DCC_)
gauge = io.gauge2ccdptzyx(gauge, params)
print("Gauge:", gauge)
print("Gauge data:", gauge.data)
print("Gauge shape:", gauge.shape)

In [ ]:
set_ptrs = np.array(params, dtype=np.int64)
print("Set pointers:", set_ptrs)
print("Set pointers data:", set_ptrs.data)
qcu.applyInitQcu(set_ptrs, params, argv)

In [12]:
# give x, b, r, r_tilde, p, v, s, t
lat_t = params[define._LAT_T_]
lat_z = params[define._LAT_Z_]
lat_y = params[define._LAT_Y_]
lat_x = int(params[define._LAT_X_]/define._LAT_P_)
lat_d = define._LAT_D_
lat_s = define._LAT_S_
lat_p = define._LAT_P_
lat_c = define._LAT_C_
latt_shape = (lat_s, lat_c, lat_t, lat_z, lat_y, lat_x)
max_iter = params[define._MAX_ITER_]
tol = argv[define._TOL_]
n = params[define._LAT_XYZT_] * define._LAT_HALF_SC_
define._LAT_Ne_ = 10
k = define._LAT_Ne_
min_eigen_value = 0
max_eigen_value = 1.6
degree = 5

In [13]:
def matvec(src):
    print("norm src", cp.linalg.norm(src))
    dest = cp.zeros(n, cp.complex64)
    qcu.applyWilsonCgDslashQcu(dest, src, gauge, set_ptrs, params)
    print("norm dest", cp.linalg.norm(dest))
    return dest


def _matvec(src, alpha, beta):
    c = (beta + alpha) / 2
    e = (beta - alpha) / 2
    return (matvec(src) - c * src) / e


def chebyshev_filter(src, alpha, beta):
    t_prev = src
    t_curr = _matvec(src, alpha, beta)
    for i in range(1, degree):
        t_next = 2 * _matvec(t_curr, alpha, beta) - t_prev
        t_prev = t_curr
        t_curr = t_next / cp.linalg.norm(t_next)
    return t_curr


def inverse_iteration_with_chebyshev():
    eigenvalues = []
    eigenvectors = []
    alpha = min_eigen_value
    beta = max_eigen_value
    for eigen_index in range(k):
        t0 = perf_counter()
        v = cp.random.randn(n) + 1j * cp.random.randn(n)
        v = v / cp.linalg.norm(v)
        v = v.astype(cp.complex64)
        if eigenvectors:
            Q = cp.column_stack(eigenvectors)
            v = v - Q @ (Q.conj().T @ v)
            v = v / cp.linalg.norm(v)
        lambda_prev = float('inf')
        for iter in range(max_iter):
            w = chebyshev_filter(v, alpha, beta)
            if eigenvectors:
                Q = cp.column_stack(eigenvectors)
                w = w - Q @ (Q.conj().T @ w)
                w = w / cp.linalg.norm(w)
            lambda_curr = float(cp.real(cp.vdot(w, matvec(w))))
            _tol = abs(lambda_curr - lambda_prev) / lambda_curr
            print("eigen index:", eigen_index, "iter:", iter, "alpha:", alpha, "beta:", beta,
                  "tol:", _tol, "lambda:", lambda_curr)
            if _tol < tol:
                break
            v = w
            lambda_prev = lambda_curr
            if iter % 10 == 0:
                alpha = max(alpha, lambda_curr * 0.9)
        eigenvalues.append(lambda_curr)
        eigenvectors.append(w)
        beta = alpha * 1.1
        t1 = perf_counter()
        print("eigen index:", eigen_index, "time:", t1-t0)
    return cp.array(eigenvalues), cp.array(eigenvectors)

In [ ]:
eigenvalues, eigenvectors = inverse_iteration_with_chebyshev()

In [ ]:
eigenvalues

In [ ]:
eigenvectors

In [ ]:
matvec(eigenvectors[0])

In [ ]:
eigenvectors[0]*eigenvalues[0]

In [ ]:
# Verify results
print("Computed eigenvalues:")
for i, ev in enumerate(eigenvalues):
    print(f"λ_{i} = {ev:.8f}")
    # Verify eigenvector
    v = eigenvectors[i]
    w = cp.zeros_like(v)
    w = matvec(v)
    error = cp.linalg.norm(w - ev * v) / cp.linalg.norm(w)
    print(f"Relative error: {error:.2e}")

In [115]:
# qcu.applyEndQcu(set_ptrs, params)